In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import random
import math

In [3]:
# Defining imshow function
def imshow(image = None, title = "Image", size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize = (size * aspect_ratio, size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

##### In out images folder first comes brain mri image and then segmented corpus callosum images. So we can get our only brain images and corpus callosum images with seperating them by odd and even indexes

In [4]:
folder_oasis_images = "oasis_imgs/oasis_imgs"
folder_abide_images = "abide_imgs/abide_imgs"
def is_even(number):
    return number % 2 != 0
oasis_images_names = [filename for filename in os.listdir(folder_oasis_images)]
oasis_cc_images = [oasis_images_names[i] for i in range(len(oasis_images_names)) if is_even(i)]
oasis_brain_images = [oasis_images_names[i] for i in range(len(oasis_images_names)) if not is_even(i)]
print("Oasis corpus callosum images: \n \n", oasis_cc_images[:5], "\n \n \n \n")
print("Oasis brain mri images: \n \n", oasis_brain_images[:5], "\n \n \n \n")

abide_images_names = [filename for filename in os.listdir(folder_abide_images)]
abide_cc_images = [abide_images_names[i] for i in range(len(abide_images_names)) if not is_even(i)]
abide_brain_images = [abide_images_names[i] for i in range(len(abide_images_names)) if is_even(i)]

print("Abide corpus callosum images: \n \n", abide_cc_images[:5], "\n \n \n \n")
print("Abide brain mri images: \n \n", abide_brain_images[:5], "\n \n \n \n")


Oasis corpus callosum images: 
 
 ['OAS1_0001_MR1_62_ccseg.tif', 'OAS1_0001_MR1_63_ccseg.tif', 'OAS1_0001_MR1_64_ccseg.tif', 'OAS1_0001_MR1_65_ccseg.tif', 'OAS1_0001_MR1_66_ccseg.tif'] 
 
 
 

Oasis brain mri images: 
 
 ['OAS1_0001_MR1_62.tif', 'OAS1_0001_MR1_63.tif', 'OAS1_0001_MR1_64.tif', 'OAS1_0001_MR1_65.tif', 'OAS1_0001_MR1_66.tif'] 
 
 
 

Abide corpus callosum images: 
 
 ['Caltech_0051456_abide_cc.tiff', 'Caltech_0051457_abide_cc.tiff', 'Caltech_0051458_abide_cc.tiff', 'Caltech_0051459_abide_cc.tiff', 'Caltech_0051460_abide_cc.tiff'] 
 
 
 

Abide brain mri images: 
 
 ['Caltech_0051456_abide_msp.tiff', 'Caltech_0051457_abide_msp.tiff', 'Caltech_0051458_abide_msp.tiff', 'Caltech_0051459_abide_msp.tiff', 'Caltech_0051460_abide_msp.tiff'] 
 
 
 



#### Now we are seperating brain images and cc images

In [5]:
def add_brain_images_to_folder(index, brain_images_array, brain_images_folder_path, file_path = "brain_images"):
    brain_image_name = brain_images_array[index]
    brain_image = cv2.imread(os.path.join(brain_images_folder_path, brain_image_name))
    file_path = f"{file_path}/{brain_image_name}"
    cv2.imwrite(file_path, brain_image)

In [6]:
image_index = 0
# for filename in range(len(oasis_brain_images)):
#     add_brain_images_to_folder(image_index, oasis_brain_images, "oasis_imgs/oasis_imgs")
#     image_index+=1
# for filename in range(len(abide_brain_images)):
#     add_brain_images_to_folder(image_index, abide_brain_images, "abide_imgs/abide_imgs")
#     image_index+=1


In [7]:
image_index = 0
# for filename in range(len(oasis_cc_images)):
#     add_brain_images_to_folder(image_index, oasis_cc_images, "oasis_imgs/oasis_imgs", "cc_images")
# #     image_index+=1
# for filename in range(len(abide_cc_images)):
#     add_brain_images_to_folder(image_index, abide_cc_images, "abide_imgs/abide_imgs", "cc_images")
#     image_index+=1

#### This functions detecting corupus callosum image in brain mri image according to segmented cc image.  Drawing rectangle around this corpus callosum image. Getting the width, height and center coordinades of this rectanlge. And normalize them between 0, 1(because this is how yolo works). Then creating txt file for each image with these coordinates like that (class, center_x, center_y, with, height. 

In [9]:
def detectPositionsOfSquare(index):
    target_cc_path = "cc_images" 
    target_brain_path = "brain_images"
    brain_images_name = os.listdir(target_brain_path)[index]
    corpus_callosum_images_name = os.listdir(target_cc_path)[index]
    target_image = os.path.join(target_cc_path, corpus_callosum_images_name)
    corpus_callosum_image = cv2.imread(target_image)
    # Grayscaling image
    corpus_callosum_image_gray = cv2.cvtColor(corpus_callosum_image, cv2.COLOR_BGR2GRAY)
    # Thresholding image
    _, th2 = cv2.threshold(corpus_callosum_image_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    corpus_callosum_image_copy = corpus_callosum_image.copy()
    # Finding and drawing contours
    contours, hierarchy = cv2.findContours(th2, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    cv2.drawContours(corpus_callosum_image_copy, contours, -1, (0, 255, 0), thickness = 2)
    
    if len(contours) > 0:
        corpus_callosum_contour = contours[0]
        x_cc, y_cc, w_cc, h_cc = cv2.boundingRect(corpus_callosum_contour)

        # Step 5: Draw the rectangle
        cv2.rectangle(corpus_callosum_image, (x_cc, y_cc), (x_cc + w_cc, y_cc + h_cc), (0, 255, 0), 2)

        # Display the image with the rectangle

    else:
        print("No contour found for the corpus callosum.")
    
    center_coordinate = (math.floor((x_cc + w_cc/2)), math.floor((y_cc + h_cc/2)))
    folder_name = "labels"
    
    base_name, extension = os.path.splitext(brain_images_name)
    txt_name = base_name + ".txt"

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    file_path = os.path.join(folder_name, txt_name)
    x_of_center, y_of_center = center_coordinate[0], center_coordinate[1]

    width, height = corpus_callosum_image.shape[0], corpus_callosum_image.shape[1]

    center_x_normalized = x_of_center / width
    center_y_normalized = y_of_center / height
    width_normalized = w_cc / width
    height_normalized = h_cc / height

    txt_writing = f'0 {center_x_normalized} {center_y_normalized} {width_normalized} {height_normalized}'
    with open(file_path, "w") as file:
        file.write(txt_writing)

In [59]:
# image_index = 0
# for filename in range(len(os.listdir("brain_images"))):
#     detectPositionsOfSquare(image_index)
#     image_index += 1

In [65]:
brain_images = os.listdir("brain_images")
cc_coordinates = os.listdir("labels")

print(f"Length of brain images folder: {len(brain_images)}")
print(f"Length of corpus callosum labels is: {len(cc_coordinates)}")

brain_images[432], cc_coordinates[432]

Length of brain images folder: 2003
Length of corpus callosum labels is: 2003


('OAS1_0001_MR1_69.tif', 'OAS1_0001_MR1_69.txt')

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.yaml")
results = model.train(data = 'config.yaml', epochs = 3) # train the model